In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('groupby and join in sql') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/03 12:20:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/03 12:20:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/03 12:20:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")


In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [9]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [13]:
df_join = spark.read.parquet('data/report/revenue/total')

In [14]:
df_join.show(vertical=True)

-RECORD 0------------------------------------
 hour                  | 2020-01-01 00:00:00 
 zone                  | 14                  
 green_amount          | null                
 green_number_records  | null                
 yellow_amount         | 8.8                 
 yellow_number_records | 1                   
-RECORD 1------------------------------------
 hour                  | 2020-01-01 00:00:00 
 zone                  | 15                  
 green_amount          | null                
 green_number_records  | null                
 yellow_amount         | 34.09               
 yellow_number_records | 1                   
-RECORD 2------------------------------------
 hour                  | 2020-01-01 00:00:00 
 zone                  | 17                  
 green_amount          | 195.03000000000003  
 green_number_records  | 9                   
 yellow_amount         | 220.20999999999998  
 yellow_number_records | 8                   
-RECORD 3-------------------------

In [18]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_zones.show()

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [21]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.locationid)

In [23]:
df_result.show(vertical=True)

-RECORD 0-------------------------------------
 hour                  | 2020-01-01 00:00:00  
 zone                  | 14                   
 green_amount          | null                 
 green_number_records  | null                 
 yellow_amount         | 8.8                  
 yellow_number_records | 1                    
 locationid            | 14                   
 borough               | Brooklyn             
 zone                  | Bay Ridge            
 service_zone          | Boro Zone            
-RECORD 1-------------------------------------
 hour                  | 2020-01-01 00:00:00  
 zone                  | 15                   
 green_amount          | null                 
 green_number_records  | null                 
 yellow_amount         | 34.09                
 yellow_number_records | 1                    
 locationid            | 15                   
 borough               | Queens               
 zone                  | Bay Terrace/Fort ... 
 service_zone

In [24]:
df_result.drop('locationid', 'zone').write.parquet('tmp/revenue-zones')